<a href="https://colab.research.google.com/github/Saivijayakumar/exeter-challenge/blob/main/Exeter_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Python Libraries

In [ ]:
import sys
import time
import re
import pandas as pd
from itertools import compress

## Reading CSV and Text Files

In [ ]:
words_data = pd.read_csv("/content/french_dictionary.csv", header=None)
words_data.index = words_data[0]
words_data.drop(labels=[0], axis=1, inplace=True)
words_data.index.name = "english"
words_data.columns = ["french"]

In [ ]:
find_words = []
file = open("/content/find_words.txt")
find_words = ["".join(lines.split()) for lines in file.readlines()]
file.close()

In [ ]:
replace = []
file = open("/content/t8.shakespeare.txt")
replace = ["".join(lines.replace("\n", "")) for lines in file.readlines()]
file.close()

## Counting Find Words Before Translating

In [ ]:
words_count_before = {word:0 for word in find_words}
freq_replace = {word:0 for word in find_words}

In [ ]:
for i in range(len(replace)):
    splitted_words = replace[i].split()
    status = [True if (i.lower() in find_words) else False for i in splitted_words]
    idx = list(compress(range(len(status)), status))
    if len(idx) > 0:
        for j in idx:
            words_count_before[splitted_words[j].lower()] = words_count_before.get(splitted_words[j].lower()) + 1
    else:
        pass

## Translating from English to French

In [ ]:
start_time = time.time()

for i in range(len(replace)):
    splitted_words = replace[i].split()
    status = [True if (i.lower() in find_words) else False for i in splitted_words]
    idx = list(compress(range(len(status)), status))
    if len(idx) > 0:
        for j in idx:
            freq_replace[splitted_words[j].lower()] = freq_replace.get(splitted_words[j].lower()) + 1
            splitted_words[j] = words_data.loc[splitted_words[j].lower()].values[0]
        replace[i] = " ".join(splitted_words)
    else:
        pass

end_time = time.time()
print("Time Taken to Translate : ", round((end_time - start_time),0), "seconds.")

Time Taken to Translate :  25.0 seconds.


## Objects Memory Size

In [ ]:
print("Size of Find Words : ", sys.getsizeof(find_words), "bytes.", "(", sys.getsizeof(find_words) / 1e+6, "in MB)")
print("Size of English to French Dictionary : ", sys.getsizeof(words_data), "bytes.", "(", sys.getsizeof(words_data) / 1e+6, "in MB)")
print("Size of Replacement Text : ", sys.getsizeof(replace), "bytes.", "(", sys.getsizeof(replace) / 1e+6, "in MB)")
total_size = (sys.getsizeof(find_words) / 1e+6) + (sys.getsizeof(words_data) / 1e+6) + (sys.getsizeof(replace) / 1e+6)
print("Total Memory Size Required : ", total_size, "MB")


Size of Find Words :  9032 bytes. ( 0.009032 in MB)
Size of English to French Dictionary :  173900 bytes. ( 0.1739 in MB)
Size of Replacement Text :  1043568 bytes. ( 1.043568 in MB)
Total Memory Size Required :  1.2265000000000001 MB


## Writing the Translated Script

In [ ]:
final = [replace[i] + "\n" for i in range(len(replace))]

In [ ]:
file = open("/content/translated_output.txt", 'w')
file.writelines(final)
file.close()

## Checking Whether All Words are Replaced

In [ ]:
status = []
for i, j in zip(words_count_before.items(), freq_replace.items()):
    if (i[1] - j[1]) != 0:
      print(i[0], "wasn't replaced completely.")
    else:
      status.append(True)

## Writing the Frequency of Words Replaced in CSV File

In [ ]:
final_res = pd.DataFrame(data=[freq_replace.keys()]).T
final_res.columns = ["English"]

In [ ]:
final_df = pd.concat([final_res, words_data["french"]], axis=1)

In [ ]:
french_words = words_data["french"]
french_words.index = range(1000)

In [ ]:
final_df = pd.concat([final_res, words_data["french"]], axis=1)

In [ ]:
final_df = pd.concat([final_df, pd.Series(freq_replace.values(), index=range(1000))], axis=1)
final_df.columns = ["English", "French", "Frequency Replaced"]

In [ ]:
final_df.to_csv("/content/replaced_frequency.csv", index=None)